In [ ]:
import urllib.request
import gzip
import os
import numpy as np
import pickle

# MNIST 데이터 다운로드
def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    files = ["train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz",
             "t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz"]
    
    for file in files:
        url = base_url + file
        print(f"Downloading {file}...")
        urllib.request.urlretrieve(url, file)
        
        with gzip.open(file, 'rb') as f_in:
            with open(file.replace(".gz", ""), 'wb') as f_out:
                f_out.write(f_in.read())
                
        os.remove(file)
        print(f"Saved {file.replace('.gz', '')}")

download_mnist()

# MNIST 데이터 로드
def load_mnist():
    def read_idx(filename):
        with open(filename, 'rb') as f:
            zero, data_type, dims = np.frombuffer(f.read(4), dtype=np.uint8)
            shape = tuple(np.frombuffer(f.read(4 * dims), dtype=np.int32))
            return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)
    
    x_train = read_idx("train-images-idx3-ubyte")
    y_train = read_idx("train-labels-idx1-ubyte")
    x_test = read_idx("t10k-images-idx3-ubyte")
    y_test = read_idx("t10k-labels-idx1-ubyte")
    
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()
x_train, x_test = x_train / 255.0, x_test / 255.0  # 정규화


In [ ]:
import numpy as np

def conv2d(input_data, kernel, bias, stride=1, padding=0):
    n, h, w, c = input_data.shape
    kh, kw, kc, nc = kernel.shape
    assert c == kc, "Kernel channels and input channels must match."
    
    padded_input = np.pad(input_data, ((0, 0), (padding, padding), (padding, padding), (0, 0)), mode='constant')
    
    out_h = (h - kh + 2 * padding) // stride + 1
    out_w = (w - kw + 2 * padding) // stride + 1
    output = np.zeros((n, out_h, out_w, nc))
    
    for i in range(out_h):
        for j in range(out_w):
            h_start, h_end = i * stride, i * stride + kh
            w_start, w_end = j * stride, j * stride + kw
            region = padded_input[:, h_start:h_end, w_start:w_end, :]
            output[:, i, j, :] = np.tensordot(region, kernel, axes=([1, 2, 3], [0, 1, 2])) + bias
    return output

def relu(x):
    return np.maximum(0, x)

def max_pooling(input_data, size=2, stride=2):
    n, h, w, c = input_data.shape
    out_h = (h - size) // stride + 1
    out_w = (w - size) // stride + 1
    output = np.zeros((n, out_h, out_w, c))
    
    for i in range(out_h):
        for j in range(out_w):
            h_start, h_end = i * stride, i * stride + size
            w_start, w_end = j * stride, j * stride + size
            region = input_data[:, h_start:h_end, w_start:w_end, :]
            output[:, i, j, :] = np.max(region, axis=(1, 2))
    return output

def dense(input_data, weights, bias):
    return np.dot(input_data, weights) + bias


In [ ]:
# 간단한 CNN 모델 구성
class SimpleCNN:
    def __init__(self):
        # Convolution layer 1 parameters
        self.conv1_kernel = np.random.randn(3, 3, 1, 8) * 0.01
        self.conv1_bias = np.zeros(8)
        
        # Convolution layer 2 parameters
        self.conv2_kernel = np.random.randn(3, 3, 8, 16) * 0.01
        self.conv2_bias = np.zeros(16)
        
        # Fully connected layer parameters
        self.fc_weights = np.random.randn(7*7*16, 10) * 0.01
        self.fc_bias = np.zeros(10)
    
    def forward(self, x):
        x = conv2d(x, self.conv1_kernel, self.conv1_bias, stride=1, padding=1)
        x = relu(x)
        x = max_pooling(x, size=2, stride=2)
        
        x = conv2d(x, self.conv2_kernel, self.conv2_bias, stride=1, padding=1)
        x = relu(x)
        x = max_pooling(x, size=2, stride=2)
        
        x = x.reshape(x.shape[0], -1)
        x = dense(x, self.fc_weights, self.fc_bias)
        
        return x

# 모델 초기화 및 예측 수행
cnn = SimpleCNN()
x_test_sample = x_test[:10, :, :, np.newaxis]  # 테스트 샘플 10개
predictions = cnn.forward(x_test_sample)
print("Predictions:", predictions)
